In [1]:
import pandas as pd, geemap, ee, json

In [2]:
ee.Authenticate() # Authenticate Earth Engine
ee.Initialize() # Initialize the API
print("EE Version:", ee.__version__) # Print Earth Engine version

EE Version: 1.6.14


__Retrieves NAIP for Meck County__


In [ ]:
# Select AOI: Mecklenburg County, NC from TIGER counties
meck = (
    ee.FeatureCollection('TIGER/2018/Counties')      # Load U.S. county boundaries
    .filter(ee.Filter.eq('NAME', 'Mecklenburg'))     # Filter by county name
    .filter(ee.Filter.eq('STATEFP', '37'))            # Restrict to North Carolina (FIPS 37)
)

def add_props(img):
    yr = ee.Date(img.get('system:time_start')).get('year')  # Extract acquisition year
    bands_str = img.bandNames().join(',')                   # Store band names as string
    return img.set({'year': yr, 'bands_str': bands_str})    # Attach metadata to image

# Load NAIP imagery intersecting Mecklenburg County
naip_all = (
    ee.ImageCollection('USDA/NAIP/DOQQ')   # NAIP aerial imagery
    .filterBounds(meck)                    # Spatial filter to AOI
    .map(add_props)                        # Add year and band metadata
)

# Get distinct NAIP acquisition years
years = (
    ee.List(naip_all.aggregate_array('year'))
    .distinct()
    .sort()
)
print('years available:', years.getInfo())

# Summarize NAIP imagery characteristics by year
def summarize(y):
    icY = naip_all.filter(ee.Filter.eq('year', y))           # Images for a single year
    total = icY.size()                                       # Total image count
    withN = icY.filter(                                      # Images containing NIR band
        ee.Filter.listContains('system:band_names', 'N')
    ).size()
    bandsets = icY.aggregate_array('bands_str').distinct().sort()  # Unique band configs
    return ee.Feature(None, {                                # Store summary as feature
        'year': y,
        'total_images': total,
        'images_with_N': withN,                              #'N' (near-infrared) band
        'band_sets': bandsets
    })

# Build per-year summary table and convert to pandas DataFrame
byyear_fc = ee.FeatureCollection(years.map(summarize))       # Server-side summary
rows = [f['properties'] for f in byyear_fc.getInfo()['features']]  # Client-side fetch
summary_df = pd.DataFrame(rows).sort_values('year')         
summary_df

__Retrieves NAIP for specific year__


In [18]:
year = 2023  # Analysis year for NAIP imagery

start = ee.Date.fromYMD(year, 1, 1)        # Start date: Jan 1 of selected year
end = start.advance(1, 'year')             # End date: Jan 1 of following year

naip_year = (ee.ImageCollection('USDA/NAIP/DOQQ')  # Load NAIP imagery collection
             .filterBounds(meck)                  # Spatial filter to Mecklenburg County AOI
             .filterDate(start, end))             # Temporal filter to selected year

print('Image count:', naip_year.size().getInfo()) # Number of NAIP images available for the year

first = naip_year.first()  # Select first image in the filtered collection
first_bands = first.bandNames().getInfo()
print('Band names of first image:', 
      first.bandNames().getInfo())                # Inspect available spectral bands

Image count: 8
Band names of first image: ['R', 'G', 'B', 'N']


__Visulization of imegery in Earth Engine__

In [6]:
# Create an interactive map centered on Charlotte with a light basemap
m = geemap.Map(center=[35.23, -80.84], zoom=10, basemap='CartoDB.Positron')

# Merge NAIP image tiles for the year and clip to Mecklenburg County boundary
mosaic = naip_year.mosaic().clip(meck)

# Add RGB visualization of the NAIP mosaic to the map
m.addLayer(
    mosaic,
    {'bands': ['R', 'G', 'B'], 'min': 0, 'max': 255, 'gamma': 1.1},
    f'NAIP RGB {year}'
)

# m

# Unsupervised Vegitation Indices

__Calcualte  NDVI, SAVI, EVI if NIR exists__

In [11]:
# Check whether Near-Infrared (NIR) band exists (needed for vegetation indices)
has_nir = 'N' in first_bands

# Initialize interactive map centered on Charlotte, NC
m = geemap.Map(center=[35.23, -80.84], zoom=11, basemap='CartoDB.Positron')

# Add NAIP RGB mosaic to map for visual reference
m.addLayer(
    mosaic,
    {'bands': ['R', 'G', 'B'], 'min': 0, 'max': 255, 'gamma': 1.1},
    f'NAIP RGB {year}'
)

# If NIR band is available, compute vegetation indices
if has_nir:
    
    # NDVI = (NIR − Red) / (NIR + Red) → basic vegetation greenness
    ndvi = mosaic.normalizedDifference(['N', 'R']).rename('NDVI')

    # Soil-Adjusted Vegetation Index (reduces soil brightness effects)
    # SAVI = ((N − R) / (N + R + L)) × (1 + L), with L = 0.5
    L = 0.5
    savi = (
        mosaic.select('N').subtract(mosaic.select('R'))
        .divide(mosaic.select('N').add(mosaic.select('R')).add(L))
        .multiply(1 + L)
        .rename('SAVI')
    )

    # Enhanced Vegetation Index (improves sensitivity in high biomass areas)
    # EVI = 2.5 × (N − R) / (N + 6R − 7.5B + 1)
    evi = (
        mosaic.select('N').subtract(mosaic.select('R'))
        .multiply(2.5)
        .divide(
            mosaic.select('N')
            .add(mosaic.select('R').multiply(6))
            .subtract(mosaic.select('B').multiply(7.5))
            .add(1)
        )
        .rename('EVI')
    )

    # Define green color palette for vegetation visualization
    green = ['#f7fcf5', '#e5f5e0', '#c7e9c0', '#74c476', '#31a354', '#006d2c']

    # Add vegetation indices to map
    m.addLayer(ndvi, {'min': 0, 'max': 1,   'palette': green}, 'NDVI')
    m.addLayer(savi, {'min': 0, 'max': 1.3, 'palette': green}, 'SAVI')
    m.addLayer(evi,  {'min': -1,'max': 1,   'palette': green}, 'EVI')

# If NIR is unavailable, notify user (RGB-only imagery)
else:
    print("This year is RGB-only (no NIR). Showing RGB-based indices below.")


# Create a multi-band feature image by selecting NAIP RGB + NIR bands
# and appending vegetation indices (NDVI, SAVI) → total of 6 bands
features = mosaic.select(['R','G','B','N']).addBands([ndvi, savi])

# Initialize an interactive map centered on the Area of Interest (AOI)
m = geemap.Map(center=[35.23, -80.84], zoom=12)

# Optional satellite basemap for visual reference (disabled here)
# m.add_basemap("SATELLITE")

# Add the NAIP RGB mosaic layer with visualization parameters
m.addLayer(
    mosaic,
    {'bands': ['R','G','B'], 'min': 0, 'max': 255},
    'NAIP RGB'
)

# Enable user-drawn geometries (e.g., polygons, points) on the map
m.add_draw_control()

# Display the interactive map
m

# Display interactive map
m

Map(center=[35.23, -80.84], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright…

## SUPERVISED Classification using Random Forest Model


In [21]:
# Create a multi-band feature image by selecting NAIP RGB + NIR bands
# and appending vegetation indices (NDVI, SAVI) → total of 6 bands
features = mosaic.select(['R','G','B','N']).addBands([ndvi, savi])

# Initialize an interactive map centered on the Area of Interest (AOI)
m = geemap.Map(center=[35.23, -80.84], zoom=12)

# Optional satellite basemap for visual reference (disabled here)
# m.add_basemap("SATELLITE")

# Add the NAIP RGB mosaic layer with visualization parameters
m.addLayer(
    mosaic,
    {'bands': ['R','G','B'], 'min': 0, 'max': 255},
    'NAIP RGB'
)

# Enable user-drawn geometries (e.g., polygons, points) on the map
m.add_draw_control()

# Display the interactive map
m


Map(center=[35.23, -80.84], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright…

In [22]:
#Check number of drawn polygons
print("Polygons drawn so far:", m.user_rois.size().getInfo())

Polygons drawn so far: 1


In [24]:
geemap.ee_export_geojson( # Save polygons to GeoJSON Forest
    ee_object=m.user_rois,
    filename=f"../../../Data/Final_dataset/vegetation_temp/forest_poi_{year}.geojson"
)
print("✅ Forest polygons saved")

✅ Forest polygons saved


**Note: Reinitialize the map for drawing the next category**

In [ ]:
geemap.ee_export_geojson( # Save polygons to GeoJSON Grass
    ee_object=m.user_rois,
    filename=f"../../../Data/Final_dataset/vegetation_temp/grass_agri_poi_{year}.geojson"
)
print("✅ Grass polygons saved")

In [ ]:
geemap.ee_export_geojson(
    ee_object=m.user_rois,
    filename=f"../../../Data/Final_dataset/vegetation_temp/others_poi_{year}.geojson"
)
print("✅ Others polygons saved")

Load Training Samples Back

In [ ]:
# --- Forest ---
with open(r"../../../Data/temp/forest_poi.geojson") as f:
    forest_json = json.load(f)
forest = geemap.geojson_to_ee(forest_json).map(lambda f: f.set("class", 0))

# --- Grass ---
with open(r"../../../Data/temp/grass_poi.geojson") as f:
    grass_json = json.load(f)
grass = geemap.geojson_to_ee(grass_json).map(lambda f: f.set("class", 1))

# --- Others (note the filename has 's') ---
with open(r"../../../Data/temp/others_poi.geojson") as f:
    others_json = json.load(f)
others = geemap.geojson_to_ee(others_json).map(lambda f: f.set("class", 2))

training_samples = forest.merge(grass).merge(others) # Merge all into one FeatureCollection

print("Forest points:", forest.size().getInfo())
print("Grass points:", grass.size().getInfo())
print("Others points:", others.size().getInfo())
print("Total training samples:", training_samples.size().getInfo())

In [ ]:
#Converts labeled geojson objects into labeled pixels
training = features.sampleRegions(
    collection=training_samples,
    properties=['class'],
    scale=1  # NAIP resolution = 1 m
)
print("Sampled training data:", training.size().getInfo())

In [ ]:
# Train by Random Forest model with 50 trees
classifier = ee.Classifier.smileRandomForest(50).train( 
    features=training,
    classProperty='class',
    inputProperties=features.bandNames()
)
classified = features.classify(classifier) # Apply classifier

In [ ]:
# Visualization
palette = ['006400', '7CFC00', '808080']  # forest=dark green, grass=light green, others=gray
m.addLayer(classified, {'min': 0, 'max': 2, 'palette': palette}, 'Classification')
m

In [ ]:
# Split training data into train/test (70/30 split)
withRandom = training.randomColumn('random')
train_set  = withRandom.filter('random < 0.7')
test_set   = withRandom.filter('random >= 0.7')

trained = ee.Classifier.smileRandomForest(50).train( # Train on training set
    features=train_set,
    classProperty='class',
    inputProperties=features.bandNames()
)

validated = test_set.classify(trained) # Validate on test set

conf_matrix = validated.errorMatrix('class', 'classification')
print("Confusion Matrix:", conf_matrix.getInfo())
print("Overall Accuracy:", conf_matrix.accuracy().getInfo())

In [ ]:
conf_matrix = validated.errorMatrix('class', 'classification') # Confusion matrix
print("Overall Accuracy:", conf_matrix.accuracy().getInfo()) # Overall accuracy
print("Kappa:", conf_matrix.kappa().getInfo()) # Kappa coefficient
print("Producer's Accuracy:", conf_matrix.producersAccuracy().getInfo()) # Producer's accuracy (recall)
print("User's Accuracy:", conf_matrix.consumersAccuracy().getInfo()) # User's accuracy (precision)

## K-FOLD Classification Cross-Validation